In [90]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

def find_problem(problem_id: int):
    api_url = f'{os.getenv("DOCKER_API_BASE_URL")}/problems/{problem_id}' if os.path.exists('/.dockerenv') else f'{os.getenv("API_BASE_URL")}/problems/{problem_id}'
    headers = { 'Content-Type': 'application/json' }
    try:
        response = httpx.get(api_url, headers=headers)
        if 200 <= response.status_code < 300:
            print(f'✅ Successfully found problem id {problem_id}')
            return response.json()
        else:
            print(f'❌ Failed to get problem id {problem_id} with status code {response.status_code}')

    except httpx.RequestError as e:
        print(f'❌ Error sending request to codescript-backend: {e}')

problem_data = find_problem(1)

user_submission = '''
    I will solve this problem using recursion, where ways(n) = ways(n-1) + ways(n-2). The base cases are ways(1) = 1 and ways(2) = 2.
'''

prompt = f'''
    ### System Instructions:
    You do not remember any past conversations. Treat this as a completely new session with no prior knowledge.

    ### Scenario:
    You are my professor specializing in coding problems. I will describe my submission for solving a problem, and you will analyze and provide structured feedback on my submission to a valid solution. Respond to me like a human, using "you" instead of "the user."

    ### My Submission (FOCUS ON THIS AND PROVIDE FEEDBACK): 
    {user_submission}

    ### Instructions:
    - **STOP**. Read and evaluate my submission before doing anything else. Your only purpose is to analyze my submission.
    - If my submission does not attempt to solve the problem, is unrelated to solving the problem, or tells you "I don't know", explicitly state that in the analysis, provide two starting suggestions, score it as 1, and **skip Problem Details.**
    - DO NOT generate explanations, reasoning, or additional insights beyond what is explicitly written in my submission.

    ### Expected Output (FOLLOW THIS EXACTLY):
    Analysis: 
        <Provide feedback on my submission in full sentences if related; otherwise, explicitly state that my submission is unrelated> 
    Suggestions: 
        - <Bullet point suggestions if related; otherwise, provide two general starting points>
    Score: 
        <
            Integer 1, 2, or 3:
            3 → The submission is valid within given constraints.
            2 → The submission is technically valid but inefficient for larger constraints or needs improvements.
            1 → The submission is fundamentally incorrect or is unrelated to solving coding problems.
        >
     
    ### Problem Details (FOR REFERENCE ONLY, IGNORE THIS IF MY SUBMISSION IS UNRELATED):
    Title: {problem_data['title']}
    Description: {problem_data['description']}
    Constraints: {''.join([f'{constraint}' for constraint in problem_data['constraints']])}
    Examples: {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}
'''

# print(prompt)


✅ Successfully found problem id 1


In [91]:
from ollama import Client

def ask_model(prompt: str):
    try:
        response = Client().generate(model='deepseek-r1:7b', prompt=prompt)
        return response['response']
    except Exception as e:
        print(f'❌ Error sending request to ollama client: {e}')

response = ask_model(prompt)


In [92]:
import re

def parse_response(response: str) -> dict:
    analysis_match = re.search(r'Analysis:\s*(.*?)(?=\nSuggestions:)', response, re.DOTALL)
    suggestions_match = re.search(r'Suggestions:\s*(.*?)(?=\nScore:)', response, re.DOTALL)
    score_match = re.search(r'Score:\s*(\d+)', response)

    analysis = analysis_match.group(1).strip() if analysis_match else None
    suggestions = [s.strip('- ') for s in suggestions_match.group(1).strip().split("\n")] if suggestions_match else []
    score = int(score_match.group(1)) if score_match else None

    return {
        'analysis': analysis,
        'suggestions': suggestions,
        'score': score
    }

res = parse_response(response)
print(res)


{'analysis': 'The submission correctly implements a recursive approach based on the Fibonacci sequence for calculating the number of distinct ways to climb stairs. However, this naive implementation is inefficient due to redundant calculations and does not handle larger values of n efficiently.', 'suggestions': ['Implement memoization or dynamic programming to store previously computed values to avoid recalculating them.', 'Consider an iterative approach if recursion becomes too slow, especially for larger n within the constraint (up to 45 steps).'], 'score': 2}
